In [2]:
# เชื่อม Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [17]:
# เตรียม data generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # แบ่ง val 20%
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

data_dir = "/content/drive/MyDrive/flowers"  # โฟลเดอร์รวมทุก class

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

val_gen = val_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True,
    seed=42
)


Found 240 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [19]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

# โหลดข้อมูล train/val/test
train_ds = image_dataset_from_directory(
    '/content/drive/MyDrive/dataset/train',
    image_size=(224, 224),
    batch_size=32
)

val_ds = image_dataset_from_directory(
    '/content/drive/MyDrive/dataset/val',
    image_size=(224, 224),
    batch_size=32
)

test_ds = image_dataset_from_directory(
    '/content/drive/MyDrive/dataset/test',
    image_size=(224, 224),
    batch_size=32
)


Found 210 files belonging to 3 classes.
Found 60 files belonging to 3 classes.
Found 30 files belonging to 3 classes.


In [20]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_ds.class_names), activation='softmax')  # จำนวน class อัตโนมัติ
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint = ModelCheckpoint(
    'flowers_model.keras',
    monitor='val_accuracy',
    save_best_only=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=7,
    callbacks=[checkpoint]
)


Epoch 1/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 1.0000 - loss: 8.1755e-04 - val_accuracy: 0.6667 - val_loss: 3.8659
Epoch 2/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.9729 - loss: 0.1604 - val_accuracy: 0.8167 - val_loss: 0.6752
Epoch 3/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.9931 - loss: 0.0361 - val_accuracy: 0.6833 - val_loss: 1.7960
Epoch 4/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9939 - loss: 0.0412 - val_accuracy: 0.7333 - val_loss: 1.1213
Epoch 5/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9974 - loss: 0.0226 - val_accuracy: 0.7000 - val_loss: 0.8928
Epoch 6/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 1.0000 - loss: 0.0067 - val_accuracy: 0.7667 - val_loss: 1.0874
Epoch 7/7
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 1.0000 - loss: 0.0090 - val_accuracy: 0.7667 - val_loss: 0.7389


In [29]:
from tensorflow.keras.models import load_model

model = load_model('flowers_model.keras')

loss, accuracy = model.evaluate(test_ds)
print(f"✅ Test Accuracy: {accuracy:.4f}")
print(f"📉 Test Loss: {loss:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 939ms/step - accuracy: 0.6667 - loss: 2.2597
✅ Test Accuracy: 0.6667
📉 Test Loss: 2.2597


In [31]:
ModelCheckpoint('flowers_model.keras', save_best_only=True)


In [32]:
import tensorflow as tf

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/dataset/test/",
    image_size=(224, 224),
    batch_size=32,
    shuffle=False  # สำหรับการประเมิน ให้ปิด shuffle จะได้แม่นยำ
)


Found 30 files belonging to 3 classes.
